In [ ]:
pip install evaluate pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00


Permite conectar la cuenta de wandb mediante una key



In [ ]:
"""!pip install wandb --quiet
import wandb
wandb.login()"""


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:


Abort: 

CODIGO PARA VISUALIZAR DATOS EN WANDB

In [ ]:
import os

CODIGO PARA ENTRENAMIENTO


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import evaluate #librería corregida para cargar métricas
import numpy as np
import torch
import warnings


# Suprimir advertencias de modelo (común en Fine-Tuning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# 1. CONFIGURACIÓN ---------
MODEL_NAME = "bert-base-uncased" #Aqui se coloca el modelo que quieras usar
OUTPUT_DIR = "./bert_modelo_propio" #Nombre de la carpeta del modelo entrenado
NUM_LABELS = 4 #4 clases: 0, 1, 2, 3
MAX_LENGTH = 128  #Define el tamaño estándar para todos los mensajes tokenizados. Si un commit es más largo, se trunca; si es más corto, se rellena con padding.
#se refiere numero maximo de tokens que el modelo BERT puede procesar en una sola entrada.

#2. CARGA Y PREPROCESAMIENTO DE DATOS --------
#aqui se coloca el dataset que se usara para el entrenamiento
NOMBRE_TRAINING_DATA = "DATASET_REFERENCIA_traducido_v2.csv"

try:
    # CORRECCIÓN para UnicodeDecodeError: Intentar codificación Latin-1
    df = pd.read_csv(NOMBRE_TRAINING_DATA, encoding='latin-1', on_bad_lines='skip')
except UnicodeDecodeError:
    # Si falla, intentar con cp1252 (común en Windows)
    df = pd.read_csv(NOMBRE_TRAINING_DATA, encoding='cp1252',on_bad_lines='skip')
except FileNotFoundError:
    print("ERROR: Asegúrate de que "+NOMBRE_TRAINING_DATA +" esté en el mismo directorio.")
    exit()

print(f"Dataset cargado exitosamente. Total de filas: {len(df)}")
#lo de arriba son configuraciones con el formato del dtaset

print(df.columns.tolist())


#aqui se coloca la columna de los mensajes, con sus etiquetas para el entrenamiento
COLUMNA_ENTRENAMIENTO = 'new_message1'
COLUMNA_ETIQUETAS = 'label'

# Seleccionar solo las columnas necesarias;las de mensajes y clasifiaicación
data = df[[COLUMNA_ENTRENAMIENTO, COLUMNA_ETIQUETAS]].copy()
# Asegurar que las etiquetas sean enteros (obligatorio para clasificación) nomas pa confirmar
data[COLUMNA_ETIQUETAS] = data[COLUMNA_ETIQUETAS].astype(int)

# Dividir el dataset de forma estratificada para manejar el desequilibrio de clases
#divide el dataset en 80% para entrenamiento y 20% para evaluación
#lo de stratify=data[COLUMNA_ETIQUETAS] asegura que la proporción de clases se mantenga en ambos conjuntos(medio raro)
train_df, eval_df = train_test_split(
    data,
    test_size=0.2,
    random_state=42, #semilla fija, para que cada vez que se ejecute agarre los mismos datos apra entrenamineto y testeo
    stratify=data[COLUMNA_ETIQUETAS] # Mantiene la proporción de etiquetas
)

#guardar el conjunto de validación (20 %)
eval_df.to_csv("validation_subset.csv", index=False, encoding="utf-8")
print(" Conjunto de validación guardado en 'validation_subset.csv'") #-------------------------------------------------------------> xsiaca se guarda

# Cargar el Tokenizer pre-entrenado. jala el tokenizador de BERT
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)


# Asegurar que todos los valores sean texto y reemplazar vacíos o nulos
train_df[COLUMNA_ENTRENAMIENTO] = train_df[COLUMNA_ENTRENAMIENTO].astype(str).fillna("")
eval_df[COLUMNA_ENTRENAMIENTO] = eval_df[COLUMNA_ENTRENAMIENTO].astype(str).fillna("")

# Eliminar filas vacías o con solo espacios
train_df = train_df[train_df[COLUMNA_ENTRENAMIENTO].str.strip() != ""]
eval_df = eval_df[eval_df[COLUMNA_ENTRENAMIENTO].str.strip() != ""]


# Función de tokenización
def tokenize_function(examples):
    return tokenizer(examples[COLUMNA_ENTRENAMIENTO], truncation=True, max_length=MAX_LENGTH)

# Convertir DataFrames a objetos Dataset de Hugging Face
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
eval_dataset = Dataset.from_pandas(eval_df.reset_index(drop=True))

# Aplicar la función de tokenización
#aca los datos de entrenamiento y evaluacion se tokenizan(se vuelven vectores numericos)
#el batched = true permite procesar varios textos a la vez, mejorando la eficiencia
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

#3. CARGA DEL MODELO Y MÉTRICAS -------

# Cargar el modelo BERT para clasificación de secuencia
#aca carga los pesos del BERT preentrenado y le añade una Capa de Clasificación Lineal con 4 salidas. Es el modelo neuronal listo para el Fine-Tuning.
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS
)

# Cargar la métrica desde la librería 'evaluate'
metric = evaluate.load("f1") #para motrar metricas f1
accuracy_metric = evaluate.load("accuracy") #para mostrar accuracy de cada epoch

# Función para calcular las métricas (Crucial para desequilibrio) después de cada ciclo de entrenamiento
#son metricas de calidad del modelo
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # F1-Score PONDERADO: Prioriza la precisión general
    f1_weighted = metric.compute(predictions=predictions, references=labels, average="weighted")
    # F1-Score MACRO: Prioriza la precisión de las clases minoritarias
    # 2. F1-Score MACRO ('macro'):
    # Es la métrica CRUCIAL para el desequilibrio. Trata a CADA CLASE (0, 1, 2, 3) por igual.
    # Si el modelo falla en detectar la clase minoritaria (ej. etiqueta 1),
    # esta puntuación baja significativamente, forzando un rendimiento alto en todas las categorías de calidad.
    f1_macro = metric.compute(predictions=predictions, references=labels, average="macro")
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)

    return {"accuracy": accuracy["accuracy"],"f1_weighted": f1_weighted['f1'], "f1_macro": f1_macro['f1']}

# 4.DEFINICIÓN DE ARGUMENTOS DE ENTRENAMIENTO ---

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="epoch",       # evalúa cada epoch
    save_strategy="epoch",
     save_total_limit=1,
    load_best_model_at_end=True,      # ya no hay checkpoints que cargar
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    report_to=["none"]                   #--------------------------------------> para mandar datos a wandb
)


# 5.INICIALIZACIÓN Y EJECUCIÓN DEL TRAINER

#Estas lineas inicializan los parametros del proyecto a donde se enviaran los datos a wandb
#import wandb
#wandb.init(project="bert-commit-quality", entity="giancarlo-villavicencio-universidad-nacional-mayor-de-sa", name="run_bert_10epochs")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

#para verificar si agarro gpu o cpu
if torch.cuda.is_available():
    print("🚀 Entrenando en GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠ Entrenando en CPU (activa GPU en Entorno de ejecución → Cambiar tipo → GPU)")

print("\n--- INICIO DEL FINE-TUNING ---")
# Iniciar el entrenamiento (Aquí se guardan los archivos en OUTPUT_DIR)
trainer.train()

print("\n--- ENTRENAMIENTO FINALIZADO ---")
# Evaluar el modelo final
results = trainer.evaluate()
print("Resultados de la Evaluación Final:")
print(results)



Dataset cargado exitosamente. Total de filas: 1597
['ï»¿id', 'repo_id', 'label', 'url', 'message', 'new_message1', 'authorEmail', 'commitDate']
 Conjunto de validación guardado en 'validation_subset.csv'


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 Entrenando en GPU: Tesla T4

--- INICIO DEL FINE-TUNING ---


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro
1,No log,1.016306,0.587500,0.448665,0.307499
2,1.212800,0.892150,0.665625,0.627497,0.485568
3,0.906800,0.791058,0.671875,0.626243,0.494778
4,0.716100,0.659422,0.743750,0.736600,0.688166
5,0.556600,0.692985,0.762500,0.764930,0.721062



--- ENTRENAMIENTO FINALIZADO ---


Resultados de la Evaluación Final:
{'eval_loss': 0.692984938621521, 'eval_accuracy': 0.7625, 'eval_f1_weighted': 0.7649300279645674, 'eval_f1_macro': 0.7210619261606104, 'eval_runtime': 2.2643, 'eval_samples_per_second': 141.321, 'eval_steps_per_second': 8.833, 'epoch': 5.0}


CODIGO PARA SACAR EL ZIP Y PODER DESCARGARLO


In [ ]:
import os

# Define la ruta de la carpeta que quieres comprimir (usando la variable OUTPUT_DIR o la ruta local)
# Asegúrate de que esta ruta sea correcta, si usaste otro nombre, cámbiala aquí:
MODEL_DIR = "./bert_modelo_propio"

# Define el nombre del archivo ZIP de salida
ZIP_FILENAME = "modelo_bertEspaniol_5epoch.zip"

# Comprime la carpeta usando el comando zip de Linux (esto debe funcionar en Colab)
# El -r incluye recursivamente todos los archivos.
!zip -r {ZIP_FILENAME} {MODEL_DIR}

print(f"Carpeta '{MODEL_DIR}' comprimida exitosamente en '{ZIP_FILENAME}'.")

  adding: bert_modelo_propio/ (stored 0%)
  adding: bert_modelo_propio/checkpoint-400/ (stored 0%)
  adding: bert_modelo_propio/checkpoint-400/config.json (deflated 53%)
  adding: bert_modelo_propio/checkpoint-400/model.safetensors (deflated 7%)
  adding: bert_modelo_propio/checkpoint-400/optimizer.pt (deflated 26%)
  adding: bert_modelo_propio/checkpoint-400/trainer_state.json (deflated 70%)
  adding: bert_modelo_propio/checkpoint-400/special_tokens_map.json (deflated 80%)
  adding: bert_modelo_propio/checkpoint-400/vocab.txt (deflated 54%)
  adding: bert_modelo_propio/checkpoint-400/tokenizer_config.json (deflated 75%)
  adding: bert_modelo_propio/checkpoint-400/rng_state.pth (deflated 26%)
  adding: bert_modelo_propio/checkpoint-400/scheduler.pt (deflated 61%)
  adding: bert_modelo_propio/checkpoint-400/training_args.bin (deflated 53%)
Carpeta './bert_modelo_propio' comprimida exitosamente en 'modelo_bertEspaniol_5epoch.zip'.
